In [1]:
import akasha
from akasha.akashas import atman, Doc_QA
from akasha.summary import Summary
import akasha.prompts as prompts
from akasha.db import RecursiveCharacterTextSplitter
import os

/home/ychm0304/anaconda3/envs/akasha/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
DEFAULT_REPORT_FORMAT = {
    "起": {},
    "承": {},
    "轉": {},
    "和": {},
}
DEFAULT_SUMMARY_FORMAT = prompts.JSON_formatter_dict(
    [
        {"name": "標題1", "description": "第一個標題和內文", "type": "str"},
        {"name": "標題2", "description": "第二個標題和內文", "type": "str"},
        {"name": "標題3", "description": "第三個標題和內文", "type": "str"},
        {"name": "標題4", "description": "第四個標題和內文", "type": "str"}
    ]
)

In [4]:
class ReportGenerator(atman):
    def __init__(
            self,
            report_format=DEFAULT_REPORT_FORMAT
    ):
        self.report_format = report_format
        self.summary = akasha.Summary()

    def _chunk_splitter(self, text):
        text_splitter = RecursiveCharacterTextSplitter(
            separators=["\n", " ", ",", ".", "。", "!"],
            chunk_size=self.chunk_size,
            chunk_overlap=self.chunk_overlap,
        )
        split_chunk = text_splitter.split_text(text)
        return split_chunk

    @staticmethod
    def _summarize_and_generate_headings(dir_path, format_prompt="", formatter=DEFAULT_SUMMARY_FORMAT):
        sum_tool = Summary()
        qa = Doc_QA()
        final_sum = ""
        json_prompt = prompts.JSON_formatter(formatter=formatter)
        for file_name in os.listdir('doc'):
            if os.path.isfile(os.path.join('doc', file_name)):
                print("##################################")
                print("Processing summarization and heading of ", file_name)
                print("##################################")
                tmp_sum = sum_tool.summarize_file(
                    dir_path,
                    system_prompt=json_prompt,
                    format_prompt=format_prompt
                )
                # tmp_sum = summarization_with_heading = qa.ask_self(
                #     info=tmp_sum,
                #     prompt="依照內文加上相對應的標題，嚴格遵守格式如下:\n\t<標題>:\n\t\t<內文>\n\t<標題>:\n\t\t<內文>\n...",
                # )
                final_sum += tmp_sum
        return final_sum
    
    def _get_content(self, file_path):
        print("##################################")
        print("Loading file name: ", os.path.basename(file_path))
        print("##################################")
        documents = akasha.db._load_file(file_path=file_path, extension=file_path.split(".")[-1])
        text_splitter = RecursiveCharacterTextSplitter(
            separators=["\n", " ", ",", ".", "。", "!"],
            chunk_size=self.summary.chunk_size, 
            chunk_overlap=self.summary.chunk_overlap,
        )
        docs = text_splitter.split_documents(documents)
        self.summary.doc_length = akasha.helper.get_docs_length(self.summary.language, docs)
        return [doc.page_content for doc in docs]

    def _single_file_title_summary(self, theme, title, texts):
        system_prompt = "你是一個非常擅長寫報告的電池產業專業人員，你會根據指定的格式來生成報告，且內容會完全符合指定的主題。若你無法回答會告訴我原因。"
        format_prompt = f"以\"{theme}\"為主題，以\"{title}\"為標題生成對應內容，不需要結論。"
        sum = self.summary.summarize_articles(
            texts,
            summary_len=250,
            system_prompt=system_prompt,
            format_prompt=format_prompt,
        )
        return sum

    def _final_title_summary(self, theme, texts):
        QA = Doc_QA()
        response = QA.ask_self(
            prompt=f"將此內容以\"{theme}\"為主題進行整合，以客觀角度撰寫，避免使用\"報告中提到\"相關詞彙，避免修改專有名詞，避免做出總結，\
                        生出約 250 字的報告。",
            info=texts,
            model="openai:gpt-4"
        )
        print(f"----------integrated summary----------")
        print(response + "\n")
        return response
    
    def generate_report(self, theme, titles, dir_path):
        report = {}
        for title in titles:
            title_summaries = []
            for file_name in os.listdir(dir_path):
                if os.path.isfile(os.path.join(dir_path, file_name)):
                    texts = self._get_content(os.path.join(dir_path, file_name))
                    title_summaries.append(
                        self._single_file_title_summary(
                            theme,
                            title,
                            texts
                        )
                    )
            report[title] = self._final_title_summary(theme, title_summaries)
        return report

In [5]:
report_gen = ReportGenerator()
print(report_gen.generate_report("電池產業發展的趨勢", ["內容摘要", "企業範圍", "產業發展動向", "企業動態", "企業前瞻"], "doc"))

##################################
Loading file name:  11.pdf
##################################


Reduce_map Summary:  68%|██████▊   | 13/19 [00:47<00:21,  3.62s/it]


這份報告的摘要確實涵蓋了臺灣燃料電池產業的潛在市場、成本優勢、中國大陸市場、應用方面的潛在用途，以及未來政策推動的建議。這些內容對於瞭解電池產業發展的趨勢非常有幫助。 




##################################
Loading file name:  43-6p13-18.pdf
##################################


Reduce_map Summary: 100%|██████████| 3/3 [00:03<00:00,  1.28s/it]


內容摘要：電池產業發展的趨勢

隨著全球對可再生能源和電動車的需求不斷增加，電池產業正迎來快速發展的趨勢。其中，鋰電池技術的進步和成本的下降，使得電動車的市場份額不斷擴大。同時，能源儲存系統的需求也在增加，以應對可再生能源的波動性。此外，新型電池技術的研發和應用也成為產業的關鍵趨勢，例如固態電池和鋰硫電池等。隨著技術的不斷創新和成本的進一步下降，電池產業將持續成為能源轉型和電動化的重要推動力。 




##################################
Loading file name:  5335cf00-58ac-456a-b1f1-f72529cedd63.pdf
##################################


...

這份摘要提供了有關臺灣電池產業發展的重要資訊。臺灣電池產業正面臨新的挑戰和機遇，包括超級電池芯工廠的建成和預計到2024年將有5座超級電芯廠量產。同時，臺灣吸引了中國大陸的人才迴流，但中國的電池產業面臨供過於求和價格崩跌的困境。臺灣企業積極參與電池產業，並計劃擴大產能，但中國在上游原材料佈局方面已取得優勢，對臺灣產業造成一定影響。儘管面臨逆境，臺灣電池產業仍以樂觀態度應對，並力求取得供應主導權。這些資訊對於企業前瞻和策略制定具有重要參考價值。 




##################################
Loading file name:  電池2.txt
##################################


Reduce_map Summary: 100%|██████████| 1/1 [00:05<00:00,  5.47s/it]


"企業前瞻：臺灣電池產業發展的趨勢"

臺灣電池產業面臨著高投資、高技術門檻及高風險的挑戰，然而在電池模組領域取得成功。要在國際市場競爭，需要內需市場配合及國家政策力量支持。預估到2025年，全球鋰電池產業總產值將超過千億美元，對電動車和綠能產業的發展將產生重大影響。臺灣的電池廠商面臨著來自中日韓和歐美國家的競爭壓力，需要不斷提升技術和降低成本以保持競爭力。臺灣的電池產業正面臨全球電池荒的挑戰，但也帶來了新的發展機遇。期待這第三波電池的浪潮，可以為臺灣的電池產業開創新局，提供更多電池領域的人才培育和就業機會，以延續國內電池產業技術發展的命脈。

這樣的內容符合您的要求嗎？ 




##################################
Loading file name:  電池3.txt
##################################


Reduce_map Summary: 100%|██████████| 3/3 [00:06<00:00,  2.18s/it]


電池產業發展的趨勢

企業前瞻

隨著全球對可再生能源和電動交通的需求不斷增加，電池產業正處於快速發展的階段。未來幾年，我們可以預見以下幾個趨勢將對電池產業產生重大影響：

1. 鋰離子電池技術的進步：鋰離子電池一直是電動車和可再生能源存儲系統的主要能量儲存技術。未來，隨著對電池能量密度、充放電速度和安全性的要求不斷提高，我們可以預期鋰離子電池技術將繼續進步，並可能出現新的材料和結構設計。

2. 電動交通的快速增長：隨著越來越多的國家和地區推動電動車的普及，電動交通將成為電池產業的主要驅動力。這將帶動對電池產品的需求增加，並促使企業加大對電池技術和生產能力的投資。

3. 可再生能源存儲需求的增加：隨著太陽能和風能等可再生能源的應用不斷擴大，對能量存儲系統的需求也將大幅增加。電池將成為這些能源系統的關鍵組件，因此電池產業將迎來新的增長機遇。

4. 環保和可持續發展的重要性：隨著社會對環保和可持續發展的重視程度不斷提高，電池產業將面臨更嚴格的環保法規和標準。企業需要加大對環保技術和生產流程的投資，以確保產品的環保性能和可持續性。

總的來說，電池產業將在未來幾年經歷快速增長，並面臨著技術創新、市場需求和環保壓力帶來的挑戰和機遇。企業應該密切關注這些趨勢，並積極調整產品結構和技術路線，以應對未來的競爭環境。 




----------integrated summary----------
在全球範疇內，電池產業正處於一個快速發展的時期，這主要得益於對可再生能源和電動交通的需求日益增長。鋰離子電池技術的進步將成為未來幾年的重要趨勢，因為這種電池一直是電動車和可再生能源存儲系統的主要能量儲存技術。隨著對電池能量密度、充放電速度和安全性的要求不斷提高，鋰離子電池技術將繼續進步，並可能出現新的材料和結構設計。

電動交通的快速增長將成為推動電池產業發展的主要力量。這將帶動對電池產品的需求增加，並促使企業加大對電池技術和生產能力的投資。同時，隨著太陽能和風能等可再生能源的應用不斷擴大，對能量存儲系統的需求也將大幅增加。電池將成為這些能源系統的關鍵組件，因此電池產業將迎來新的增長機遇。

此外，環保和可持續發展的重要性也將對電池產業產生影響。隨著社會對環保和可持續發展的重視程度不斷提高，電池產業將面臨更嚴格的環保法規和標準。企業需要加大對環保技術和生產流程的投資，以確保產